In [13]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time
from itertools import product, combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from plotting import newfig, savefig
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
from matplotlib import rc

np.random.seed(1234)


In [2]:
def plot_solution(X_star, u_star, index):
    
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    y = np.linspace(lb[1], ub[1], nn)
    X, Y = np.meshgrid(x,y)
    
    U_star = griddata(X_star, u_star.flatten(), (X, Y), method='cubic')
    
    plt.figure(index)
    plt.pcolor(X,Y,U_star, cmap = 'jet')
    plt.colorbar()
    
    
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/4
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)

In [3]:
def load_model(model, path, device):
    model.load_state_dict(torch.load(path, map_location=device))


In [15]:
# for NS equation
from train import PhysicsInformedNN
from torch.autograd import grad
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(torch.__version__)
data = scipy.io.loadmat('./data/cylinder_nektar_wake.mat')
U_star = data['U_star'] # N x 2 x T; full data of (u, v) at all x, y, t
P_star = data['p_star'] # N x T; full data of p at all x, y, t
t_star = data['t'] # T x 1; t corrdinates of the full data
X_star = data['X_star'] # N x 2; (x,y) coordinates of the full data

N = X_star.shape[0] # N=5000; size of the full data at a fixed t
T = t_star.shape[0] # T=200; size of the full data at a fixed x,y

# Rearrange Data 
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T

UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T

x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1

u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1

# Training Data   
N_train = 5000 
idx = np.random.choice(N*T, N_train, replace=False) # Generate a random sample from np.arange(N*T) of size N_train without replacement
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]
# print(np.max(u_train), np.min(u_train), np.max(v_train),  np.min(v_train))

# Load Model
xyt_train = np.concatenate([x, y, t], 1)
pinn = PhysicsInformedNN(xyt_train, u_train, v_train, layers=8, device=device).to(device) 
print(pinn)
path = './model/20220424_1600'
load_model(pinn, path, device)


# Test Data
snap = np.array([100])
x_star = X_star[:,0:1]
y_star = X_star[:,1:2]
t_star = TT[:,snap]

u_star = U_star[:,0,snap]
v_star = U_star[:,1,snap]
p_star = P_star[:,snap]

xyt_star = np.concatenate([x_star, y_star, t_star], 1)

# Make prediction with trained model
xyt = torch.FloatTensor(xyt_star)
xyt.requires_grad=True 
u_pred, v_pred, p_pred, _, _ = pinn.forward(xyt)
# u_pred = u_pred.detach().cpu().numpy()
# v_pred = v_pred.detach().cpu().numpy()
# p_pred = p_pred.detach().cpu().numpy()

lambda_1_value = pinn.lambda_1
lambda_2_value = pinn.lambda_2

Du = grad(u_pred, xyt, grad_outputs=torch.ones_like(torch.Tensor(u_pred)), create_graph=True, retain_graph=True)
u_x = Du[0][:,0]
Du_x = grad(u_x, xyt, grad_outputs=torch.ones_like(torch.Tensor(u_pred)), create_graph=True, retain_graph=True)
u_xx = Du_x[0][:,0]
Du_xx = grad(u_xx, xyt, grad_outputs=torch.ones_like(torch.Tensor(u_pred)), create_graph=True, retain_graph=True)
u_xxx = Du_xx[0][:,0]
Du_xxx = grad(u_xxx, xyt, grad_outputs=torch.ones_like(torch.Tensor(u_pred)), create_graph=True, retain_graph=True)
u_xxxx = Du_xxx[0][:,0]

print(torch.linalg.norm(u_x, 2))
print(torch.linalg.norm(u_xx, 2))
print(torch.linalg.norm(u_xxx, 2))
print(torch.linalg.norm(u_xxxx, 2))
    

1.11.0+cu102
PhysicsInformedNN(
  (input_layer): Linear(in_features=3, out_features=20, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=20, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Linear(in_features=20, out_features=20, bias=True)
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): Linear(in_features=20, out_features=20, bias=True)
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): Linear(in_features=20, out_features=20, bias=True)
  )
  (ouput_layer): Linear(in_features=20, out_features=2, bias=True)
)
torch.Size([5000]) torch.Size([5000, 3])


/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor(16.8779, grad_fn=<CopyBackwards>)
tensor(31.7202, grad_fn=<CopyBackwards>)
tensor(93.4944, grad_fn=<CopyBackwards>)
tensor(403.3717, grad_fn=<CopyBackwards>)


In [17]:
# for heat equation
from train_heat import PhysicsInformedNN, load_data
from torch.autograd import grad
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
collc_xt, collc_z, data_xt, data_z, test_xt, test_z = load_data(device)

optim_method = "adam"
layers = 8
nIter = 50000  # original niter is 200000
lr = 0.001
batch = 10000

# Load Model
pinn = PhysicsInformedNN(data_xt, data_z, layers, device, optim_method, lr).to(device) 
print(pinn)
path = './model/20220426_1719'
load_model(pinn, path, device)

# Make prediction with trained model
test_xt = test_xt[:2]
u_pred, _ = pinn.forward(test_xt)
u_pred = u_pred.squeeze()
print(u_pred.shape, test_xt.shape)

Du = grad(u_pred, test_xt, grad_outputs=torch.ones_like(torch.Tensor(u_pred)), create_graph=True, retain_graph=True)
u_x = Du[0][:,0]
Du_x = grad(u_x, test_xt, grad_outputs=torch.ones_like(torch.Tensor(u_pred)), create_graph=True, retain_graph=True)
u_xx = Du_x[0][:,0]
Du_xx = grad(u_xx, test_xt, grad_outputs=torch.ones_like(torch.Tensor(u_pred)), create_graph=True, retain_graph=True)
u_xxx = Du_xx[0][:,0]
Du_xxx = grad(u_xxx, test_xt, grad_outputs=torch.ones_like(torch.Tensor(u_pred)), create_graph=True, retain_graph=True)
u_xxxx = Du_xxx[0][:,0]

print(torch.linalg.norm(u_x, 2))
print(torch.linalg.norm(u_xx, 2))
print(torch.linalg.norm(u_xxx, 2))
print(torch.linalg.norm(u_xxxx, 2))
    

PhysicsInformedNN(
  (input_layer): Linear(in_features=2, out_features=20, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=20, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Linear(in_features=20, out_features=20, bias=True)
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): Linear(in_features=20, out_features=20, bias=True)
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): Linear(in_features=20, out_features=20, bias=True)
  )
  (ouput_layer): Linear(in_features=20, out_features=1, bias=True)
)
torch.Size([2]) torch.Size([2, 2])
tensor(4.2734, grad_fn=<CopyBackwards>)
tensor(11.5721, grad_fn=<CopyBackwards>)
tensor(147.6738, grad_fn=<CopyBackwards>) tensor([121.8490, -83.4289], grad_fn=<SelectBackward0>)
tensor(334.2044, grad_fn=<CopyBackwards>)


/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
